<a href="https://colab.research.google.com/github/elhamod/BA820/blob/main/Association_Rules/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Course: BA820 - Unsupervised and Unstructured ML**

**Notebook created by: Mohannad Elhamod**

#Actionable Movie Viewing Patterns

In this notebook, we will reason about patterns in audience behavior. By observing how movies, genres, and user preferences co-occur in viewing and preference histories, we can answer practical business questions that sit at the core of recommendation systems, content acquisition, and user growth.

## Data Loading

In [3]:
# supressing some unimportant warnings
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [4]:
import io
import zipfile
import requests
import pandas as pd

# Official MovieLens 100K URL
url = "https://files.grouplens.org/datasets/movielens/ml-100k.zip"

# Download the zip
resp = requests.get(url)
resp.raise_for_status()

# Open zip from bytes
z = zipfile.ZipFile(io.BytesIO(resp.content))

# -----------------------------
# Ratings (u.data)
# -----------------------------
with z.open("ml-100k/u.data") as f:
    ratings = pd.read_csv(
        f,
        sep="\t",
        names=["user_id", "item_id", "rating", "timestamp"],
        engine="python",
    )

# -----------------------------
# Movies (u.item)
# -----------------------------
with z.open("ml-100k/u.item") as f:
    movies = pd.read_csv(
        f,
        sep="|",
        header=None,
        encoding="latin-1",
        names=[
            "movie_id",
            "title",
            "release_date",
            "unknown",
            "Action",
            "Adventure",
            "Animation",
            "Children's",
            "Comedy",
            "Crime",
            "Documentary",
            "Drama",
            "Fantasy",
            "Film-Noir",
            "Horror",
            "Musical",
            "Mystery",
            "Romance",
            "Sci-Fi",
            "Thriller",
            "War",
            "Western",
        ],
    )

# -----------------------------
# Users (u.user)
# -----------------------------
with z.open("ml-100k/u.user") as f:
    users = pd.read_csv(
        f,
        sep="|",
        names=["user_id", "age", "gender", "occupation", "zip_code"],
    )


# -----------------------------
# Preview the dataframes
# -----------------------------
print("RATINGS:")
display(ratings)

print("MOVIES:")
display(movies)

print("USERS:")
display(users)



RATINGS:


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


MOVIES:


,,movie_id,title,release_date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


USERS:


,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [5]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

##Questions:

### Assuming you work for *Netflix*, you are to tackle the following questions and problems...

###Q1. Implement the following feature: When the user browses a specific movie, calculate how likely they are to watch it given their *viewing history* ?

###Q2. For users who strongly prefer a given genre, which other genres should be suppressed from homepage carousels and personalized collections to avoid disengaging recommendations?

###Q3. Given that a user likes a specific movie, which movies should we show in the 'Because You Liked…' carousel to maximize follow-through engagement?

###Q4. If a viewer watches many Romantic Comedies, which top three genres are they most likely to also enjoy? Does the answer fundamentally change if we know the user's gender?

###Q5. The streaming platform has acquired The *Fellowship of the Ring*. Should we license The *Two Towers* based on observed viewer behavior?

###Q6. A new user joins the platform. How many movies must they rate during their onboarding before we can confidently recommend at least one movie they are 80% likely to enjoy?

###Q7. Which movies act as 'gateway titles' that lead viewers into *new genres*?